In [1]:
import sys
sys.path.append("../library")

In [2]:
import math
# import columns.py for csv processing
from columns import read_csv, write_csv
from __future__ import print_function, division
from thinkbayes2 import Suite, Pmf, Mean, MeanVar, EvalPoissonPmf, \
    MakeCdfFromList, MakeNormalPmf, MakeMixture, MakePoissonPmf, MakeExponentialPmf, MakeUniformPmf, \
    PmfProbLess, PmfProbEqual
import thinkplot

# Exercise 1
If buses arrive at a bus stop every 20 minutes, and you arrive at the bus stop at a random time, your wait time until the bus arrives is uniformly distributed from 0 to 20 minutes.

But in reality, there is variability in the time between buses. Suppose you are waiting for a bus, and you know **the historical distribution of time between buses.**
Compute your distribution of wait times.

Hint: Suppose that the time between buses is either 5 or 10 minutes with equal probability. What is the probability that you arrive during one of the 10 minute intervals?

In [3]:
# supposed the historical distribution of time between buses is pmf_zb
def PmfOfWaitTime(pmf_zb: Pmf):
    """
    To estimate waiting time distribution based on distribution of time between buses.
    Arguments:
        1, pmf_zb: the historical distribution of time between buses
    Return:
        distribution of waiting time of buses
    """
    metapmf = Pmf()
    for gap, prob in pmf_zb.Items():
        uniform = MakeUniformPmf(0, gap)
        metapmf.Set(uniform, prob)
    pmf_y = MakeMixture(metapmf)
    return pmf_y